# Inteligência Artificial – Projeto 02

## Grupo 16

- Tiago Carneiro — nº 28002
- Alexandre Salyha — nº 27998

### Notebook 3 – Regras de associação (Apriori)

- Unidade Curricular: Inteligência Artificial  
- Ano letivo: 2025/2026  
- Tema: Padrões nos resultados de Fórmula 1

**Objetivo deste notebook:**  
Extrair regras de associação entre pilotos que tendem a terminar nas mesmas corridas em posições de topo (por exemplo, Top 10), utilizando o algoritmo Apriori e analisando medidas como suporte, confiança e lift, de forma a identificar padrões relevantes.

### Introdução

Este notebook corresponde ao **Notebook 3 – Regras de associação** do Projeto 02 da unidade curricular de Inteligência Artificial.

O objetivo deste trabalho é aplicar técnicas de *Data Mining* para **descobrir padrões de coocorrência** em dados de Fórmula 1, recorrendo ao algoritmo **Apriori** para a extração de regras de associação. Ao contrário dos notebooks anteriores, que abordaram problemas de classificação (Notebook 1) e clustering (Notebook 2), aqui o foco está na identificação de relações do tipo:

> “Quando um determinado conjunto de pilotos aparece bem classificado numa corrida, quais são os outros pilotos que tendem a aparecer também?”

Através deste tipo de análise, é possível detetar padrões interessantes sobre a presença conjunta de pilotos em posições de destaque, o que pode refletir dinâmicas de competitividade entre equipas, épocas ou combinações de pilotos.

Ao longo deste notebook serão descritos o objetivo de negócio associado às regras de associação, o processo de preparação dos dados em formato transacional, a aplicação do algoritmo Apriori (incluindo o ajuste de parâmetros como *min_support* e *min_confidence*) e a análise dos resultados obtidos.

### Objetivo de negócio

Do ponto de vista de negócio, o problema pode ser formulado da seguinte forma:

> **Conseguir identificar combinações de pilotos que tendem a aparecer juntos nas posições cimeiras das corridas de Fórmula 1, utilizando regras de associação.**

Este tipo de conhecimento pode ser útil para:
- analisar **padrões de coocorrência** no top 10 de cada corrida;
- perceber que pilotos e equipas frequentemente competem entre si em posições semelhantes;
- apoiar análises históricas sobre rivalidades ou “grupos” de pilotos que aparecem juntos em bons resultados.

Para isso, cada **corrida** é tratada como uma **transação**, e cada **piloto que terminou no top 10** dessa corrida é tratado como um **item**. Desta forma, o algoritmo Apriori pode ser aplicado para extrair regras do tipo:

> *{Piloto A, Piloto B} → {Piloto C}*

interpretadas como: quando os pilotos A e B aparecem no top 10 de uma corrida, é frequente o piloto C também aparecer.

### Importar bibliotecas

In [1]:
# Importação de bibliotecas principais
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

from mlxtend.frequent_patterns import apriori, association_rules

sns.set_theme()

### Carregar datasets

In [2]:
# Carregar os ficheiros necessários
lap_times = pd.read_csv("../dataset/lap_times.csv")
drivers = pd.read_csv("../dataset/drivers.csv")

print("lap_times:", lap_times.shape)
print("drivers  :", drivers.shape)

display(lap_times.head())
display(drivers.head())

lap_times: (613516, 6)
drivers  : (864, 9)


,raceId,driverId,lap,position,time,milliseconds
0,841,20,1,1,1:38.109,98109
1,841,20,2,1,1:33.006,93006
2,841,20,3,1,1:32.713,92713
3,841,20,4,1,1:32.803,92803
4,841,20,5,1,1:32.342,92342


,driverId,driverRef,number,code,forename,surname,dob,nationality,url
0,1,hamilton,44,HAM,Lewis,Hamilton,1985-01-07,British,http://en.wikipedia.org/wiki/Lewis_Hamilton
1,2,heidfeld,\N,HEI,Nick,Heidfeld,1977-05-10,German,http://en.wikipedia.org/wiki/Nick_Heidfeld
2,3,rosberg,6,ROS,Nico,Rosberg,1985-06-27,German,http://en.wikipedia.org/wiki/Nico_Rosberg
3,4,alonso,14,ALO,Fernando,Alonso,1981-07-29,Spanish,http://en.wikipedia.org/wiki/Fernando_Alonso
4,5,kovalainen,\N,KOV,Heikki,Kovalainen,1981-10-19,Finnish,http://en.wikipedia.org/wiki/Heikki_Kovalainen


### Preparação dos dados para regras de associação

Para aplicar regras de associação, é necessário transformar os dados num **formato transacional**, em que cada linha representa uma transação (corrida) e cada coluna representa um item (piloto presente no top 10 dessa corrida).

O primeiro passo consiste em obter, para cada corrida, a **posição final** de cada piloto, a partir dos registos de voltas em `lap_times.csv`. Para isso, considera-se a última volta registada para cada par *(corrida, piloto)* e usa-se a respetiva posição nessa volta como aproximação da posição final.

De seguida, são selecionados apenas os pilotos que terminaram **no top 10** de cada corrida. Opcionalmente, junta-se informação de `drivers.csv` (por exemplo, o nome do piloto), para que as regras de associação fiquem mais legíveis.

In [3]:
# Garantir que temos as colunas necessárias
lap_times[['raceId', 'driverId', 'lap', 'position']].head()

# Obter a última volta de cada (raceId, driverId)
# Ordenamos por lap e ficamos com a última entrada de cada piloto em cada corrida
lap_times_sorted = lap_times.sort_values(by=["raceId", "driverId", "lap"])

last_lap_per_driver = (
    lap_times_sorted
    .groupby(["raceId", "driverId"])
    .tail(1)  # última linha de cada grupo = última volta registada
    .copy()
)

last_lap_per_driver = last_lap_per_driver[["raceId", "driverId", "position"]]

print(last_lap_per_driver.shape)
display(last_lap_per_driver.head())

(11466, 3)


,raceId,driverId,position
342938,1,1,4
343151,1,2,11
343636,1,3,7
343209,1,4,6
343653,1,6,4


De seguida, são selecionados apenas os pilotos que terminaram **no top 10** de cada corrida. Opcionalmente, junta-se informação de `drivers.csv` (por exemplo, o nome do piloto), para que as regras de associação fiquem mais legíveis.

In [4]:
# Filtrar pilotos que terminaram no top 10
top_n = 10
final_results_top = last_lap_per_driver[last_lap_per_driver["position"] <= top_n].copy()

print("Registos (raceId, driverId) no top 10:", final_results_top.shape[0])

# Juntar nome do piloto para facilitar leitura
drivers_small = drivers[["driverId", "forename", "surname"]].copy()
drivers_small["driver_name"] = drivers_small["forename"] + " " + drivers_small["surname"]

final_results_top = final_results_top.merge(drivers_small[["driverId", "driver_name"]],
                                            on="driverId",
                                            how="left")

display(final_results_top.head())

Registos (raceId, driverId) no top 10: 6483


,raceId,driverId,position,driver_name
0,1,1,4,Lewis Hamilton
1,1,3,7,Nico Rosberg
2,1,4,6,Fernando Alonso
3,1,6,4,Kazuki Nakajima
4,1,7,9,Sébastien Bourdais


### Construção da matriz transacional

Para aplicar o algoritmo Apriori, é necessário construir uma matriz binária em que:

- cada linha representa uma **corrida** (`raceId`);
- cada coluna representa um **piloto** (`driver_name`);
- o valor é **1** se o piloto terminou no top 10 dessa corrida, e **0** caso contrário.

Esta matriz corresponde a uma representação típica de *market basket*, onde cada corrida é um “carrinho de compras” e os pilotos são os “itens”.

In [5]:
# Criar matriz corrida x piloto (one-hot encoding manual)
basket = (
    final_results_top
    .groupby(["raceId", "driver_name"])["position"]
    .count()  # apenas para ter algo para unstack; o valor é irrelevante, apenas a presença conta
    .unstack(fill_value=0)
)

# Converter contagens em 0/1 (presença ou ausência)
basket_bool = basket > 0

print("Formato da matriz transacional:", basket_bool.shape)
display(basket_bool.head())

Formato da matriz transacional: (566, 118)


driver_name,Adrian Sutil,Alessandro Zanardi,Alex Yoong,Alexander Albon,Alexander Wurz,Allan McNish,Andrea Kimi Antonelli,Andrea Montermini,Antonio Giovinazzi,Antônio Pizzonia,...,Tarso Marques,Tiago Monteiro,Timo Glock,Toranosuke Takagi,Ukyo Katayama,Valtteri Bottas,Vitaly Petrov,Vitantonio Liuzzi,Yuki Tsunoda,Zsolt Baumgartner
raceId,,,,,,,,,,,,,,,,,,,,,
1,True,False,False,False,False,False,False,False,False,False,...,False,False,True,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,True,False,False,False,False,False,False,False
3,True,False,False,False,False,False,False,False,False,False,...,False,False,True,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,True,False,False,False,False,False,False,False
5,False,False,False,False,False,False,False,False,False,False,...,False,False,True,False,False,False,False,False,False,False


### Aplicação do algoritmo Apriori

Após a construção da matriz transacional, em que cada linha representa uma **corrida** (`raceId`) e cada coluna indica a presença ou ausência de um **piloto** no top 10 dessa corrida, foi aplicado o algoritmo **Apriori** para identificar conjuntos frequentes de pilotos.

No total, a matriz resultante contém:

- **566 corridas** (transações);
- **118 pilotos** considerados como itens (pilotos que apareceram pelo menos uma vez no top 10).

O Apriori foi configurado com um valor mínimo de suporte de **2%** (`min_support = 0.02`). Isto significa que apenas são considerados conjuntos de pilotos que surgem em simultâneo no top 10 de, pelo menos, cerca de:

[0.02 × 566 ≈ 11 corridas]

Com esta configuração, foram encontrados **19 653 itemsets frequentes**, isto é, combinações de pilotos que aparecem juntos em pelo menos 2% das corridas analisadas. A maioria destes itemsets é constituída por um número reduzido de pilotos (1, 2 ou 3), o que é natural, dado que conjuntos muito grandes tendem a ter um suporte reduzido.

A partir destes conjuntos frequentes, foram depois geradas **regras de associação**, recorrendo à função `association_rules`. Foi utilizada a **confiança** (*confidence*) como métrica base, com um limiar mínimo de **0.4** (`min_threshold = 0.4`), e as regras foram posteriormente filtradas de forma a manter apenas aquelas com **lift > 1.0**, ou seja, regras que indicam uma associação positiva entre os pilotos do antecedente e os pilotos do consequente.

Após a filtragem, obtiveram-se **304 178 regras de associação**, que serão analisadas e resumidas na secção seguinte.

In [6]:
# Ver quantas corridas temos
num_races = basket_bool.shape[0]
print("Número de corridas (transações):", num_races)

# Aplicar Apriori para encontrar conjuntos frequentes de pilotos
frequent_itemsets = apriori(
    basket_bool,
    min_support=0.02,      # pelo menos em 2% das corridas (ajusta se der poucos/demais resultados)
    use_colnames=True
)

# Acrescentar a dimensão (nº de itens) de cada conjunto
frequent_itemsets["length"] = frequent_itemsets["itemsets"].apply(len)

print("Número de itemsets frequentes encontrados:", len(frequent_itemsets))
display(frequent_itemsets.head())

Número de corridas (transações): 566
Número de itemsets frequentes encontrados: 19653


,support,itemsets,length
0,0.061837,(Adrian Sutil),1
1,0.090106,(Alexander Albon),1
2,0.074205,(Alexander Wurz),1
3,0.028269,(Andrea Kimi Antonelli),1
4,0.021201,(Antonio Giovinazzi),1


### Geração de regras de associação

A partir dos conjuntos frequentes de pilotos obtidos com o Apriori, são geradas **regras de associação** do tipo:

[ {A, B} → {C} ]

em que:
- o lado esquerdo (**antecedente**) representa um conjunto de pilotos que aparece no top 10;
- o lado direito (**consequente**) representa outro piloto que tende a aparecer juntamente com os anteriores.

Para cada regra são calculadas várias métricas, entre as quais:

- **support (suporte)** – proporção de corridas em que todos os pilotos da regra (antecedente ∪ consequente) aparecem em simultâneo;
- **confidence (confiança)** – probabilidade de o consequente aparecer, dado que o antecedente aparece;
- **lift** – mede o aumento relativo da probabilidade de observar o consequente quando o antecedente ocorre, comparando com o que seria esperado se fossem independentes (valores superiores a 1 indicam associação positiva).

São retidas apenas regras com um valor mínimo de confiança e de *lift*, de forma a focar a análise em padrões com algum grau de fiabilidade e interesse.

In [7]:
# Gerar regras de associação a partir dos itemsets frequentes
rules = association_rules(
    frequent_itemsets,
    metric="confidence",
    min_threshold=0.4   # confiança mínima de 40% (ajusta se necessário)
)

# Filtrar regras mais interessantes (por exemplo, lift > 1 e pelo menos 2 itens no total)
rules = rules[rules["lift"] > 1.0].copy()
rules["num_antecedent"] = rules["antecedents"].apply(len)
rules["num_consequent"] = rules["consequents"].apply(len)
rules["num_total_items"] = rules["num_antecedent"] + rules["num_consequent"]

print("Número de regras após filtragem:", len(rules))

# Ver as regras mais fortes ordenadas por lift
rules_sorted = rules.sort_values(by="lift", ascending=False)

display(
    rules_sorted[
        ["antecedents", "consequents", "support", "confidence", "lift"]
    ].head(20)
)

Número de regras após filtragem: 304178


,antecedents,consequents,support,confidence,lift
243883,"(Eddie Irvine, Kimi Räikkönen, Michael Schumac...","(Jacques Villeneuve, Jarno Trulli, Juan Pablo ...",0.021201,0.923077,17.415385
243885,"(Jacques Villeneuve, Jarno Trulli, Juan Pablo ...","(Eddie Irvine, Kimi Räikkönen, Michael Schumac...",0.021201,0.400000,17.415385
280501,"(Jacques Villeneuve, Heinz-Harald Frentzen, Je...","(Giancarlo Fisichella, Michael Schumacher, Dam...",0.021201,0.705882,17.370844
280626,"(Giancarlo Fisichella, Michael Schumacher, Dam...","(Jacques Villeneuve, Heinz-Harald Frentzen, Je...",0.021201,0.521739,17.370844
280623,"(Michael Schumacher, Alexander Wurz, Damon Hill)","(Jacques Villeneuve, Heinz-Harald Frentzen, Je...",0.021201,1.000000,17.151515
280276,"(Giancarlo Fisichella, Mika Häkkinen, Damon Hill)","(Heinz-Harald Frentzen, Jean Alesi, Alexander ...",0.022968,0.541667,17.032407
280204,"(Heinz-Harald Frentzen, Jean Alesi, Alexander ...","(Giancarlo Fisichella, Mika Häkkinen, Damon Hill)",0.022968,0.722222,17.032407
243866,"(Jacques Villeneuve, Jarno Trulli, David Coult...","(Eddie Irvine, Michael Schumacher, Kimi Räikkö...",0.021201,0.444444,16.770370
243870,"(Jacques Villeneuve, Jarno Trulli, Michael Sch...","(Eddie Irvine, David Coulthard, Juan Pablo Mon...",0.021201,0.444444,16.770370
243895,"(Eddie Irvine, Michael Schumacher, Kimi Räikkö...","(Jacques Villeneuve, Jarno Trulli, David Coult...",0.021201,0.800000,16.770370


### Resultados intermédios das regras de associação

As regras de associação geradas descrevem padrões do tipo:

\[
\{\text{Piloto(s) no antecedente}\} \Rightarrow \{\text{Piloto(s) no consequente}\}
\]

em que:

- o **antecedente** é um conjunto de pilotos que aparece no top 10 de uma corrida;
- o **consequente** é outro conjunto de pilotos que tende a aparecer juntamente com eles.

Com as configurações adotadas (`min_support = 0.02`, `confidence ≥ 0.4` e `lift > 1`), foram obtidas **304 178 regras**, o que mostra que existem muitos padrões de coocorrência entre pilotos nas corridas analisadas.

Algumas observações gerais sobre as regras mais fortes (ordenadas por *lift*):

- Muitas das regras de maior *lift* envolvem pilotos historicamente relevantes, como **Michael Schumacher, Damon Hill, Mika Häkkinen, Jacques Villeneuve, Giancarlo Fisichella, Eddie Irvine**, entre outros.  
- As regras de topo tendem a envolver **grupos de 3 ou mais pilotos** que aparecem em conjunto no top 10 de um número significativo de corridas, sugerindo que esses pilotos competiram frequentemente entre si em posições cimeiras, possivelmente em épocas ou equipas semelhantes.
- Os valores de **suporte** das regras mais fortes situam-se, em geral, na ordem dos **2% a 3%**, o que corresponde a cerca de 11 a 17 corridas em que a combinação completa (antecedente ∪ consequente) se verifica. Embora, em termos absolutos, estes valores possam parecer baixos, são relevantes num contexto histórico com muitas corridas e elevada diversidade de pilotos.
- Os valores de **confiança** para as melhores regras situam-se frequentemente acima de **0.5**, podendo em alguns casos atingir valores próximos de **0.8** ou superiores, o que indica que, quando os pilotos do antecedente aparecem no top 10, é bastante provável que os pilotos do consequente também apareçam.

De forma geral, estas regras captam **padrões de coocorrência entre pilotos que partilharam épocas, equipas ou níveis de competitividade semelhantes**, refletindo rivalidades e grupos de pilotos que apareciam juntos com frequência nas posições da frente.

### Resultados e considerações finais

Neste notebook foram aplicadas **regras de associação** a dados históricos de Fórmula 1, tratando cada corrida como uma **transação** e cada piloto que terminou no **top 10** como um **item**. A partir da informação de `lap_times.csv` e `drivers.csv`, foi construída uma matriz transacional com **566 corridas** e **118 pilotos**, permitindo aplicar o algoritmo **Apriori** e extrair padrões de coocorrência entre pilotos.

Com um suporte mínimo de 2%, foram encontrados **19 653 itemsets frequentes**, que deram origem a **304 178 regras de associação** após a aplicação de filtros baseados em confiança (*confidence*) e *lift*. As regras com *lift* mais elevado evidenciam conjuntos de pilotos que tendem a aparecer juntos nas posições cimeiras, muitas vezes correspondendo a:

- pilotos que competiram na mesma época e lutavam regularmente pelos mesmos lugares;
- combinações de pilotos associados a equipas dominantes em determinados períodos;
- grupos de pilotos que partilharam vários pódios ou presenças consistentes no top 10.

Embora os valores de suporte das regras individuais sejam, em muitos casos, relativamente baixos (da ordem dos 2–3%), isso é expectável num contexto com muitas corridas e elevada rotatividade de pilotos. Ainda assim, as regras encontradas permitem **identificar padrões estáveis de coocorrência**, sobretudo entre pilotos com carreiras mais longas ou que disputaram vários campeonatos em simultâneo.

Algumas limitações importantes a destacar são:

- A análise considera apenas a **presença no top 10**, ignorando a posição exata (1.º, 2.º, 3.º, etc.), o que simplifica o problema mas faz perder alguma granularidade.
- Não é feita distinção entre **épocas diferentes**, pelo que pilotos que nunca chegaram a competir diretamente podem não aparecer associados, mesmo que tenham desempenhos semelhantes em períodos distintos.
- A grande quantidade de regras geradas torna necessário aplicar filtros adicionais (por exemplo, restringir o tamanho dos antecedentes ou focar apenas regras com *lift* mais elevado) para facilitar a interpretação.

Como trabalho futuro, seria interessante:

- restringir a análise a **janelas temporais** específicas (por exemplo, por década ou por regulamento técnico), para obter regras mais contextuais;
- incluir informação sobre **equipas** e **épocas**, permitindo analisar não só coocorrência de pilotos, mas também padrões relacionados com construtores;
- explorar outras medidas de interesse para regras de associação (como *leverage* ou *conviction*) e comparar os resultados.

Em síntese, o Notebook 3 mostra como técnicas de *Data Mining* baseadas em regras de associação podem ser utilizadas para **descobrir padrões de coocorrência entre pilotos** na Fórmula 1, complementando as perspetivas supervisionada (Notebook 1) e não supervisionada (Notebook 2) exploradas anteriormente.